In [1]:
from nltk import pos_tag
import pandas as pd
import numpy as np 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
import nltk 
from itertools import chain
from wordcloud import WordCloud, STOPWORDS
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import seaborn as sns
import pyLDAvis
import pyLDAvis.gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models.ldamodel import LdaModel
import spacy
from pprint import pprint
nltk.download('punkt')
nltk.download('stopwords')
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

C:\Users\giorg\anaconda3\lib\site-packages\sklearn\decomposition\_lda.py:28: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  EPS = np.finfo(np.float).eps
C:\Users\giorg\anaconda3\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()
C:\Users\giorg\anaconda3\lib\site-packages\tensorflow\core\framework\tensor_shape_pb2.py:18: DeprecationWarning: Call to deprecated create function FileDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or qu

C:\Users\giorg\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:513: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  np.object,
C:\Users\giorg\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:521: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  np.bool,
C:\Users\giorg\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:555: DeprecationWarning: `np.object` is a depreca

In [2]:
pd.set_option('display.max_colwidth', None)
df = pd.read_csv(r'C:\Users\giorg\clean_text.csv', encoding = 'utf-8-sig')

# Pre-Processing

In [3]:
text = df['text']
text = list(text)


In [4]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

text_words = list(sent_to_words(text))

print(text_words[:1])

[['folks', 'paste', 'cytokine', 'storm', 'pfizerbiontech']]


In [5]:
bigram = gensim.models.Phrases(text_words, min_count=1, threshold=10) 
trigram = gensim.models.Phrases(bigram[text_words], threshold=10)  

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

print(trigram_mod[bigram_mod[text_words[0]]])

['folks', 'paste', 'cytokine', 'storm', 'pfizerbiontech']


In [6]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [7]:
# Form Bigrams
text_words_bigrams = make_bigrams(text_words)


nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
text_lemmatized = lemmatization(text_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(text_lemmatized[:1])

[['folk', 'paste', 'cytokine', 'storm', 'pfizerbiontech']]


In [8]:
id2word = corpora.Dictionary(text_lemmatized)

# Create Corpus
texts = text_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)]]


In [9]:
id2word[0]

'cytokine'

In [10]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('cytokine', 1),
  ('folk', 1),
  ('paste', 1),
  ('pfizerbiontech', 1),
  ('storm', 1)]]

# Basic LDA model

In [11]:
lda_model = LdaModel(corpus=corpus,
                     id2word=id2word,
                     num_topics=12, 
                     random_state=42,
                     update_every=1,
                     chunksize=100,
                     passes=10,
                     alpha='symmetric',
                     per_word_topics=True)

In [12]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.141*"coronavirus" + 0.063*"shot" + 0.057*"thank" + 0.028*"vaccinate" + '
  '0.027*"great" + 0.024*"start" + 0.021*"team" + 0.019*"lab" + 0.018*"drug" + '
  '0.017*"free"'),
 (1,
  '0.258*"covidvaccine" + 0.037*"supply" + 0.032*"st" + 0.031*"modi" + '
  '0.021*"president" + 0.020*"mrna" + 0.020*"pfizerbiontech" + 0.019*"work" + '
  '0.016*"bad" + 0.012*"next"'),
 (2,
  '0.160*"third" + 0.094*"news" + 0.071*"govt" + 0.034*"update" + '
  '0.029*"good_new" + 0.028*"import" + 0.027*"datum" + 0.027*"hope" + '
  '0.025*"regulator" + 0.024*"china"'),
 (3,
  '0.229*"vaccine" + 0.162*"covid" + 0.138*"covaxin" + 0.022*"available" + '
  '0.019*"indian" + 0.017*"emergency_authorisation" + 0.013*"report" + '
  '0.012*"covidindia" + 0.010*"countries_astrazeneca" + 0.008*"do"'),
 (4,
  '0.128*"amp" + 0.089*"day" + 0.062*"good" + 0.037*"week" + 0.029*"trial" + '
  '0.027*"many" + 0.023*"goi" + 0.018*"due" + 0.018*"eua" + 0.013*"tuesday"'),
 (5,
  '0.154*"moderna" + 0.093*"vaccine" + 0.083*"em

In [13]:
pprint(lda_model.top_topics(corpus,topn=10))

[([(0.2706249, 'india'),
   (0.089370444, 'russia'),
   (0.08478737, 'case'),
   (0.068225645, 'covid'),
   (0.040214133, 'vaccination'),
   (0.03181857, 'shoot'),
   (0.028200394, 'country'),
   (0.017420236, 'jab'),
   (0.013825594, 'first'),
   (0.013399612, 'emergency_approval')],
  -6.21264208978417),
 ([(0.3287762, 'sputnikv'),
   (0.16994344, 'vaccine'),
   (0.054966863, 'russian'),
   (0.052590095, 'sputnik'),
   (0.026778331, 'covishield'),
   (0.019940494, 'dr_reddys'),
   (0.011266177, 'emergency_india'),
   (0.010144842, 'need'),
   (0.009984205, 'russias'),
   (0.008681514, 'efficacy')],
  -8.03844839629743),
 ([(0.25799018, 'covidvaccine'),
   (0.036570158, 'supply'),
   (0.032114916, 'st'),
   (0.030791186, 'modi'),
   (0.020534033, 'president'),
   (0.019949771, 'mrna'),
   (0.019623805, 'pfizerbiontech'),
   (0.01860462, 'work'),
   (0.016408099, 'bad'),
   (0.012020362, 'next')],
  -8.865839641313917),
 ([(0.12780787, 'amp'),
   (0.08931354, 'day'),
   (0.061992355, '

# Model Evaluation

In [14]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=text_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.48633234106898415


# Result Visualiaztion

In [15]:
pyLDAvis.enable_notebook()
lda_visualization = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
pyLDAvis.display(lda_visualization)

In [16]:
stop = stopwords.words('english') 

cols = [color for name, color in mcolors.TABLEAU_COLORS.items()]  

cloud = WordCloud(stopwords=stop,
                  background_color='white',
                  width=2500,
                  height=1800,
                  max_words=100,
                  colormap='tab10',
                  color_func=lambda *args, **kwargs: cols[i],
                  prefer_horizontal=1.0)

topics = lda_model.show_topics(formatted=False,num_words=30)

fig, axes = plt.subplots(5, 1, figsize=(10,20), sharex=True, sharey=True)

for i, ax in enumerate(axes.flatten()):
    fig.add_subplot(ax)
    topic_words = dict(topics[i][1])
    cloud.generate_from_frequencies(topic_words, max_font_size=500)
    plt.gca().imshow(cloud)
    plt.gca().set_title('Topic ' + str(i + 1), fontdict=dict(size=16))
    plt.gca().axis('off')


plt.subplots_adjust(wspace=0, hspace=0)
plt.axis('off')
plt.margins(x=0, y=0)
plt.tight_layout()
plt.show()

# Finding ideal number of topics

In [17]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ---------- 
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model= LdaModel(corpus=corpus, id2word=id2word, num_topics=num_topics, random_state=42, update_every=1, chunksize=100, passes=10, alpha='symmetric', per_word_topics=True)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=text_lemmatized, dictionary=id2word, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())
    return model_list, coherence_values

In [18]:
start=2; limit=25; step=2;

model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=text_lemmatized, start=start, limit=limit, step=step)

# Show graph

x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.title('Coherence Score Evaluation for LDA models')
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()